<a href="https://colab.research.google.com/github/JotaMTB/Coursera_Capstone/blob/master/IBM%20Capstone%20Project%20-%20The_sweet_decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Capstone Project

IBM Data Science on Coursera

# The sweet decision
A story about how to find the best location for a new bakery.

In [ ]:
import pandas as pd
import numpy as np

DataFrame creation from wikipedia web:

In [ ]:
url_madrid = 'https://es.wikipedia.org/wiki/Anexo:Barrios_administrativos_de_Madrid'

madrid_data = pd.read_html(url_madrid)
df_madrid = pd.DataFrame(madrid_data[0])

print(df_madrid.shape)
df_madrid.head()

(131, 5)


,Distrito,Número,Nombre,Superficie (km²)[2]​,Imagen
0,Centro,11,Palacio,"1,471 km²",NaN
1,Centro,12,Embajadores,"1,032 km²",NaN
2,Centro,13,Cortes,"0,592 km²",NaN
3,Centro,14,Justicia,"0,742 km²",NaN
4,Centro,15,Universidad,"0,947 km²",NaN


DataFrame cleaning:

In [ ]:
df_madrid.drop(['Imagen', 'Número'], axis=1, inplace=True)
df_madrid.head()

,Distrito,Nombre,Superficie (km²)[2]​
0,Centro,Palacio,"1,471 km²"
1,Centro,Embajadores,"1,032 km²"
2,Centro,Cortes,"0,592 km²"
3,Centro,Justicia,"0,742 km²"
4,Centro,Universidad,"0,947 km²"


In [ ]:
df_madrid.rename(columns={'Distrito':'Borough', 'Nombre':'Neighborhood', 'Superficie (km²)[2]​':'Area (km^2)'}, inplace=True)
df_madrid.head()

,Borough,Neighborhood,Area (km^2)
0,Centro,Palacio,"1,471 km²"
1,Centro,Embajadores,"1,032 km²"
2,Centro,Cortes,"0,592 km²"
3,Centro,Justicia,"0,742 km²"
4,Centro,Universidad,"0,947 km²"


In [ ]:
df_madrid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Borough       131 non-null    object
 1   Neighborhood  131 non-null    object
 2   Area (km^2)   131 non-null    object
dtypes: object(3)
memory usage: 3.2+ KB


Collecting the URLs from every neighborhood to extract the coordinates (latitude and logitude):

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

r = urlopen("https://es.wikipedia.org/wiki/Anexo:Barrios_administrativos_de_Madrid")
bs = BeautifulSoup(r.read(), "html.parser")
r.close()

url_list = []

for link in bs.find_all("a"):
    #print(link.get("href"))
    url_list.append(link.get("href"))

url_list_mad = url_list[15:316]

url_list_mad[0:5]

['/wiki/Palacio_(Madrid)',
 '/wiki/Archivo:Barrio_de_Palacio_(11),_Madrid.svg',
 '/wiki/Embajadores_(Madrid)',
 '/wiki/Archivo:Barrio_de_Embajadores_(12),_Madrid.svg',
 '/wiki/Cortes_(Madrid)']

* I have exported the list '**url_list_mad**'.
* Then, I have cleaned the list using a text editor.
* Finally, I have upload the list to GitHub.
* Now, I am going to import de list:

In [ ]:
url_list = 'https://raw.githubusercontent.com/JotaMTB/Archivos_auxiliares/master/url_wiki_madrid_neighborhoods.csv'

neighborhoods_imported = pd.read_csv(url_list, header=None)
print(neighborhoods_imported.shape)
neighborhoods_imported.head()

(131, 1)


,0
0,/wiki/Palacio_(Madrid)
1,/wiki/Embajadores_(Madrid)
2,/wiki/Cortes_(Madrid)
3,/wiki/Justicia_(Madrid)
4,/wiki/Universidad_(Madrid)


In [ ]:
neighborhoods_list = neighborhoods_imported[0].tolist()

for i, url in enumerate(neighborhoods_list):
  neighborhoods_list[i] = url.strip()

print(len(neighborhoods_list))
neighborhoods_list[0:5]

131


['/wiki/Palacio_(Madrid)',
 '/wiki/Embajadores_(Madrid)',
 '/wiki/Cortes_(Madrid)',
 '/wiki/Justicia_(Madrid)',
 '/wiki/Universidad_(Madrid)']

Extract latitude and longitude from webs. Then, add the information to the DataFrame:

In [ ]:
url_path = 'https://es.wikipedia.org'

coor_list = []

for i in range(len(neighborhoods_list)):
  url = url_path + neighborhoods_list[i]
  url_data = pd.read_html(url)
  df = pd.DataFrame(url_data[0])

  if df.shape[0] > 1:
    pass
  else:
    df = pd.DataFrame(url_data[1])

  column_list = df.columns.tolist()
  column_1_list = df[column_list[0]].tolist()
  column_2_list = df[column_list[1]].tolist()

  for n in range(len(column_1_list)):
    if column_1_list[n] == 'Ubicación' or column_1_list[n] == 'Coordenadas':
      coordinate = column_2_list[n]
      break

  coor_list.append(coordinate)

In [ ]:
latitude_list = []
longitude_list = []

for coordenates in coor_list:
  coors = coordenates.split(' ')
  latitude = coors[-2].replace(',', '')
  latitude = float(latitude[1:-1])
  longitude = float(coors[-1])

  latitude_list.append(round(latitude, 4))
  longitude_list.append(round(longitude, 4))

In [ ]:
print(len(longitude_list), len(latitude_list))

131 131


In [ ]:
df_madrid['Latitude'] = latitude_list
df_madrid['Longitude'] = longitude_list

df_madrid.head(5)

,Borough,Neighborhood,Area (km^2),Latitude,Longitude
0,Centro,Palacio,"1,471 km²",40.4100,-3.7133
1,Centro,Embajadores,"1,032 km²",40.4089,-3.6997
2,Centro,Cortes,"0,592 km²",40.4142,-3.6981
3,Centro,Justicia,"0,742 km²",40.4239,-3.6964
4,Centro,Universidad,"0,947 km²",40.4253,-3.7083


Import the dataset of population in Madrid:

In [ ]:
url_population = 'https://raw.githubusercontent.com/JotaMTB/Archivos_auxiliares/master/People%20living%20in%20Madrid.csv'

df_people = pd.read_csv(url_population, sep=';', encoding='ISO-8859-1')

print(df_people.shape)
df_people.head()

(131, 2)


,Neighborhood,People
0,PALACIO,23695
1,EMBAJADORES,47224
2,CORTES,10793
3,JUSTICIA,18092
4,UNIVERSIDAD,33540


In [ ]:
df_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Neighborhood  131 non-null    object
 1   People        131 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


Now, I am going to merge the dataframes:

In [ ]:
people = df_people['People'].tolist()

df_madrid['People'] = people
df_madrid.head()

,Borough,Neighborhood,Area (km^2),Latitude,Longitude,People
0,Centro,Palacio,"1,471 km²",40.4100,-3.7133,23695
1,Centro,Embajadores,"1,032 km²",40.4089,-3.6997,47224
2,Centro,Cortes,"0,592 km²",40.4142,-3.6981,10793
3,Centro,Justicia,"0,742 km²",40.4239,-3.6964,18092
4,Centro,Universidad,"0,947 km²",40.4253,-3.7083,33540


In [ ]:
df_madrid['People'] = df_madrid['People'].astype('float')
df_madrid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Borough       131 non-null    object 
 1   Neighborhood  131 non-null    object 
 2   Area (km^2)   131 non-null    object 
 3   Latitude      131 non-null    float64
 4   Longitude     131 non-null    float64
 5   People        131 non-null    float64
dtypes: float64(3), object(3)
memory usage: 6.3+ KB


In [ ]:
df_madrid.head()

,Borough,Neighborhood,Area (km^2),Latitude,Longitude,People
0,Centro,Palacio,"1,471 km²",40.4100,-3.7133,23695.0
1,Centro,Embajadores,"1,032 km²",40.4089,-3.6997,47224.0
2,Centro,Cortes,"0,592 km²",40.4142,-3.6981,10793.0
3,Centro,Justicia,"0,742 km²",40.4239,-3.6964,18092.0
4,Centro,Universidad,"0,947 km²",40.4253,-3.7083,33540.0


### Visualization neighborhoods of Madrid:

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

lat_madrid = 40.4167
lon_madrid = -3.7039

map_madrid_neig = folium.Map(location=[lat_madrid, lon_madrid], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_madrid['Latitude'], df_madrid['Longitude'], df_madrid['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid_neig)  
    
map_madrid_neig

In [ ]:
df_madrid_filter = df_madrid[(df_madrid['Borough']=='Fuencarral-El Pardo') |
                             (df_madrid['Borough']=='Hortaleza') |
                             (df_madrid['Borough']=='Barajas')]

print(df_madrid_filter.shape)
df_madrid_filter.head(20)

(19, 6)


,Borough,Neighborhood,Area (km^2),Latitude,Longitude,People
43,Fuencarral-El Pardo,El Pardo,"187,642 km²",40.5167,-3.7667,3486.0
44,Fuencarral-El Pardo,Fuentelarreina,"1,382 km²",40.4819,-3.7425,3315.0
45,Fuencarral-El Pardo,Peñagrande,"2,890 km²",40.4781,-3.7256,45071.0
46,Fuencarral-El Pardo,El Pilar,"1,359 km²",40.4782,-3.7115,46980.0
47,Fuencarral-El Pardo,La Paz,"2,159 km²",40.4814,-3.6969,33717.0
48,Fuencarral-El Pardo,Valverde,"8,977 km²",40.5058,-3.6858,64845.0
49,Fuencarral-El Pardo,Mirasierra,"6,975 km²",40.4917,-3.7081,34077.0
50,Fuencarral-El Pardo,El Goloso,"26,509 km²",40.5561,-3.7033,19125.0
100,Hortaleza,Palomas,"1,128 km²",40.4536,-3.6181,6857.0
101,Hortaleza,Piovera,"3,139 km²",40.4536,-3.6403,14987.0


In [ ]:
import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe
import requests

CLIENT_ID = 'UTXOJCETPDJ4PK15Y3GTZKYJBW3QGDNZZ3BZUWTLE5FRKXQM' # your Foursquare ID
CLIENT_SECRET = 'KO55EJG4VENN22YORYLOGQP305QBKXZO2K3AZBVWLY5HCEGG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UTXOJCETPDJ4PK15Y3GTZKYJBW3QGDNZZ3BZUWTLE5FRKXQM
CLIENT_SECRET:KO55EJG4VENN22YORYLOGQP305QBKXZO2K3AZBVWLY5HCEGG


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, population, radius=400, LIMIT=150):
    
    venues_list=[]
    for name, lat, lng, popu in zip(names, latitudes, longitudes, population):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            popu, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'Neighborhood Population', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
madrid_venues = getNearbyVenues(names = df_madrid_filter['Neighborhood'],
                                   latitudes = df_madrid_filter['Latitude'],
                                   longitudes = df_madrid_filter['Longitude'],
                                    population = df_madrid_filter['People'])

El Pardo
Fuentelarreina
Peñagrande
El Pilar
La Paz
Valverde
Mirasierra
El Goloso
Palomas
Piovera
Canillas
Pinar del Rey
Apóstol Santiago
Valdefuentes
Alameda de Osuna
Aeropuerto
Casco Histórico de Barajas
Timón
Corralejos


In [ ]:
print(madrid_venues.shape)
madrid_venues.head()

(153, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Neighborhood Population,Venue,Venue Latitude,Venue Longitude,Venue Category
0,El Pardo,40.5167,-3.7667,3486.0,Lar de Domingo,40.514241,-3.764157,Spanish Restaurant
1,El Pardo,40.5167,-3.7667,3486.0,El Faro del Pardo,40.514210,-3.764297,Spanish Restaurant
2,Fuentelarreina,40.4819,-3.7425,3315.0,Burger King,40.481028,-3.746482,Fast Food Restaurant
3,Fuentelarreina,40.4819,-3.7425,3315.0,CD FUNDACION,40.481758,-3.738439,Soccer Field
4,Peñagrande,40.4781,-3.7256,45071.0,El Viejo Almacén de Buenos Aires,40.481124,-3.723373,Argentinian Restaurant


In [ ]:
madrid_venues_list = madrid_venues['Venue Category'].value_counts()
madrid_venues_list[0:20]

Spanish Restaurant      14
Restaurant               9
Coffee Shop              7
Fast Food Restaurant     6
Breakfast Spot           5
Hotel                    5
Tapas Restaurant         5
Asian Restaurant         5
Italian Restaurant       5
Park                     4
Sandwich Place           4
Airport Lounge           4
Bakery                   4
Café                     4
Soccer Field             4
Grocery Store            3
Plaza                    3
Supermarket              3
Burger Joint             3
Metro Station            3
Name: Venue Category, dtype: int64

In [ ]:
df_madrid_bakery = madrid_venues[madrid_venues['Venue Category'] == 'Bakery']
df_madrid_bakery

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Neighborhood Population,Venue,Venue Latitude,Venue Longitude,Venue Category
31,El Pilar,40.4782,-3.7115,46980.0,Panadería El Guerrero,40.477575,-3.713825,Bakery
46,Palomas,40.4536,-3.6181,6857.0,Mallorca,40.455062,-3.620675,Bakery
94,Apóstol Santiago,40.4750,-3.6623,15613.0,Piccolo Cafetería / Panadería,40.476767,-3.659182,Bakery
95,Alameda de Osuna,40.4572,-3.5878,19885.0,La 28,40.457335,-3.585759,Bakery


In [ ]:
df_madrid_sorted = df_madrid_filter.sort_values(['People'], ascending=False)
df_madrid_north_top10_popu = df_madrid_sorted.head(10)
df_madrid_north_top10_popu

,Borough,Neighborhood,Area (km^2),Latitude,Longitude,People
48,Fuencarral-El Pardo,Valverde,"8,977 km²",40.5058,-3.6858,64845.0
105,Hortaleza,Valdefuentes,"16,778 km²",40.4906,-3.6239,62078.0
103,Hortaleza,Pinar del Rey,"2,632 km²",40.4647,-3.6556,53344.0
46,Fuencarral-El Pardo,El Pilar,"1,359 km²",40.4782,-3.7115,46980.0
45,Fuencarral-El Pardo,Peñagrande,"2,890 km²",40.4781,-3.7256,45071.0
102,Hortaleza,Canillas,"2,516 km²",40.4597,-3.6456,40940.0
49,Fuencarral-El Pardo,Mirasierra,"6,975 km²",40.4917,-3.7081,34077.0
47,Fuencarral-El Pardo,La Paz,"2,159 km²",40.4814,-3.6969,33717.0
126,Barajas,Alameda de Osuna,"1,961 km²",40.4572,-3.5878,19885.0
50,Fuencarral-El Pardo,El Goloso,"26,509 km²",40.5561,-3.7033,19125.0


In [ ]:
lat_madrid_north = 40.4814
lon_madrid_north = -3.6969

map_madrid_north = folium.Map(location=[lat_madrid_north, lon_madrid_north], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_madrid_north_top10_popu['Latitude'], df_madrid_north_top10_popu['Longitude'], df_madrid_north_top10_popu['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid_north)

for lat, lng, label in zip(df_madrid_bakery['Neighborhood Latitude'], df_madrid_bakery['Neighborhood Longitude'], df_madrid_bakery['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid_north) 
    
map_madrid_north

## CLUSTERING:

In [ ]:
print('There are {} uniques categories.'.format(len(madrid_venues['Venue Category'].unique())))

There are 65 uniques categories.


In [ ]:
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
madrid_onehot['Neighborhood'] = madrid_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]


madrid_grouped = madrid_onehot.groupby('Neighborhood').mean().reset_index()


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = madrid_grouped['Neighborhood']

for ind in np.arange(madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Aeropuerto,Airport Lounge,Spanish Restaurant,Duty-free Shop,Café,Cafeteria,Sandwich Place,Fast Food Restaurant
1,Alameda de Osuna,Shop & Service,Smoke Shop,Metro Station,Italian Restaurant,Chinese Restaurant,Hotel,Pizza Place
2,Apóstol Santiago,Grocery Store,Metro Station,Bakery,Flea Market,Department Store,Diner,Donut Shop
3,Canillas,Hotel,Metro Station,Plaza,Restaurant,Rock Club,Spanish Restaurant,Soccer Field
4,Casco Histórico de Barajas,Hotel,Pizza Place,Spanish Restaurant,Plaza,Restaurant,Sandwich Place,Boarding House


In [ ]:
from sklearn.cluster import KMeans

kclusters = 3

madrid_grouped_clustering = madrid_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

labels_k = kmeans.labels_

In [ ]:
neighborhoods_venues_sorted['Cluster'] = labels_k
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster
0,Aeropuerto,Airport Lounge,Spanish Restaurant,Duty-free Shop,Café,Cafeteria,Sandwich Place,Fast Food Restaurant,0
1,Alameda de Osuna,Shop & Service,Smoke Shop,Metro Station,Italian Restaurant,Chinese Restaurant,Hotel,Pizza Place,0
2,Apóstol Santiago,Grocery Store,Metro Station,Bakery,Flea Market,Department Store,Diner,Donut Shop,0
3,Canillas,Hotel,Metro Station,Plaza,Restaurant,Rock Club,Spanish Restaurant,Soccer Field,0
4,Casco Histórico de Barajas,Hotel,Pizza Place,Spanish Restaurant,Plaza,Restaurant,Sandwich Place,Boarding House,0
5,Corralejos,Spanish Restaurant,Hostel,Mexican Restaurant,Coffee Shop,Restaurant,Park,Supermarket,0
6,El Pardo,Spanish Restaurant,Flea Market,Cosmetics Shop,Department Store,Diner,Donut Shop,Duty-free Shop,2
7,El Pilar,Fast Food Restaurant,Tapas Restaurant,Burger Joint,Video Game Store,Pizza Place,Frozen Yogurt Shop,General Entertainment,0
8,Fuentelarreina,Soccer Field,Fast Food Restaurant,Video Game Store,Cosmetics Shop,Department Store,Diner,Donut Shop,1
9,La Paz,Park,Food & Drink Shop,Athletics & Sports,Spanish Restaurant,Italian Restaurant,Fast Food Restaurant,Cosmetics Shop,0


In [ ]:
bakery_hoods = df_madrid_bakery['Neighborhood'].tolist()
bakery_hoods

['El Pilar', 'Palomas', 'Apóstol Santiago', 'Alameda de Osuna']

In [ ]:
neighborhoods_venues_sorted[(neighborhoods_venues_sorted['Neighborhood'] == 'El Pilar') |
                            (neighborhoods_venues_sorted['Neighborhood'] == 'Palomas') |
                            (neighborhoods_venues_sorted['Neighborhood'] == 'Apóstol Santiago') |
                            (neighborhoods_venues_sorted['Neighborhood'] == 'Alameda de Osuna')]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster
1,Alameda de Osuna,Shop & Service,Smoke Shop,Metro Station,Italian Restaurant,Chinese Restaurant,Hotel,Pizza Place,0
2,Apóstol Santiago,Grocery Store,Metro Station,Bakery,Flea Market,Department Store,Diner,Donut Shop,0
7,El Pilar,Fast Food Restaurant,Tapas Restaurant,Burger Joint,Video Game Store,Pizza Place,Frozen Yogurt Shop,General Entertainment,0
11,Palomas,Coffee Shop,Restaurant,Café,Asian Restaurant,Spanish Restaurant,Sandwich Place,Food Truck,0


All the neighborhoods with a bakery belong to the same cluster.